In [233]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [234]:
def loc_id_cnt(num):
    df = pd.read_csv('./data/%02d.csv'%num)
    df_res = df.groupby(['time_stamp', 'loc_id']).agg({ 'phone_id':'count'})
    df_res.reset_index(inplace=True)
    return df_res

In [235]:
df = loc_id_cnt(10)

In [236]:
def handle(df):
    # 时间维度
    df['datetime'] = df.time_stamp.apply(pd.to_datetime)
    del df['time_stamp']
    #df['month'] = df.datetime.apply(lambda x: x.month)
    #df['day'] = df.datetime.apply(lambda x: x.day)
    df['hour'] = df.datetime.apply(lambda x: x.hour)
    df['weekday'] = df.datetime.apply(lambda x: x.weekday())
    #df['weekend'] = df.datetime.apply(lambda x: x.weekday()>4)
    # 加入假期，节假日
    """
    def isSummer(x):
        if x > '2017-07-15 00:00:00' and x < '2017-09-03:00:00:00':
            return 1
        else:
            return 0

    def isSummerNear(x):
        if x > '2017-07-12 00:00:00' and x < '2017-07-15:00:00:00':
            return 1
        elif x > '2017-09-03 00:00:00' and x < '2017-09-06:00:00:00'::
            return 0 
    """
    def isHoliday(x):
        if x.date() == datetime.date(2017, 4, 4) or \
           x.date() == datetime.date(2017, 5, 1) or \
           x.date() == datetime.date(2017, 5, 30) or \
           x.date() == datetime.date(2017, 1, 1) or \
           x.date() >= datetime.date(2017, 10, 1) and x.date() <= datetime.date(2017, 10, 8):
            return 1
        else:
            return 0

    #df['isHoliday'] = df.datetime.apply(isHoliday)
    return df

In [237]:
df = handle(df)

In [238]:
df = df[df.datetime>'2017-10-09 00:00:00']

In [239]:
df.head()

,loc_id,phone_id,datetime,hour,weekday
6308,3,32,2017-10-09 01:00:00,1,0
6309,4,46,2017-10-09 01:00:00,1,0
6310,8,12,2017-10-09 01:00:00,1,0
6311,10,5,2017-10-09 01:00:00,1,0
6312,12,21,2017-10-09 01:00:00,1,0


In [240]:
loc_df = []
for i in range(1, 34):
    _df = df[df.loc_id==i]
    _df.reset_index(inplace=True, drop=True)
    loc_df.append(_df)
pre_df = []

In [241]:
def handle_each_loc(n, pre_df):
    loc1 = loc_df[n-1]
    stamp = datetime.datetime(2017, 11, 1, 0, 0, 0)
    while stamp < datetime.datetime(2017, 12, 1, 0, 0, 0):

        wekdy = stamp.weekday()
        tmp = loc1[loc1.weekday==wekdy]
        tmp = tmp[tmp.hour==stamp.hour]
        try:
            phn_id = round(tmp.phone_id[-2:].mean())
        except Exception as e:
            print(e)
            print(n, stamp.weekday(), stamp.hour, phn_id)
            print(stamp)
            stamp = stamp + datetime.timedelta(hours=1)
            continue
        #print(stamp.weekday(), stamp.hour, phn_id)

        append_df = pd.DataFrame()
        append_df['loc_id'] = [n]
        append_df['phone_id'] = [phn_id]
        append_df['datetime'] = [stamp]
        append_df['hour'] = [stamp.hour]
        append_df['weekday'] = [stamp.weekday()]

        loc1 = loc1.append(append_df)

        stamp = stamp + datetime.timedelta(hours=1)
    _ = loc1[loc1.datetime >= '2017-01-01 00:00:00']
    pre_df.append(_)
    return pre_df

In [242]:
for n in range(1, 34):
    pre_df = handle_each_loc(n, pre_df)

cannot convert float NaN to integer
24 4 3 1
2017-11-03 03:00:00
cannot convert float NaN to integer
24 4 3 1
2017-11-10 03:00:00
cannot convert float NaN to integer
24 4 3 1
2017-11-17 03:00:00
cannot convert float NaN to integer
24 4 3 1
2017-11-24 03:00:00


In [243]:
res = pd.DataFrame()
for each in pre_df:
    each['time_stamp'] = each['datetime'].apply(lambda x: x.strftime("%Y-%m-%d %H"))
    each['num_of_people'] = each['phone_id']
    res = res.append(each[['loc_id','time_stamp','num_of_people']])
res = res[res.time_stamp>='2017-11-01 00']

In [244]:
res = res.sort_values(by=['time_stamp','loc_id'])

In [245]:
res.to_csv('res.csv')

In [246]:
len(res)

23756

In [247]:
df[df]

ValueError: Must pass DataFrame with boolean values only